# 02 - Price Data Preprocessing

## Objective
Transform raw price data into modeling-ready format.

**Key Tasks:**
1. Load and clean data
2. Feature engineering (46 features)
3. Handle negative prices (keep for now, analyze impact)
4. Train/Val/Test split
5. Scaling (StandardScaler)
6. Save processed datasets

**Special Considerations for Price:**
- Negative prices are valid (oversupply) → keep them
- High volatility → lag features crucial
- Spike detection features may help

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Load Data

In [ ]:
# Load raw data
data_path = Path('../../data/raw/price_day_ahead_2022-01-01_2024-12-31_hour.csv')
df = pd.read_csv(data_path)

# Parse datetime
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)
df.sort_index(inplace=True)
df.columns = ['price']

print(f"Loaded: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
df.head()

## 2. Data Quality Checks

In [ ]:
print("Missing values:", df.isnull().sum().sum())
print(f"Negative prices: {(df['price'] < 0).sum()} (keeping them as valid data)")
print(f"Zero prices: {(df['price'] == 0).sum()}")

# Check for duplicates
print(f"Duplicate timestamps: {df.index.duplicated().sum()}")

## 3. Feature Engineering (46 Features)

Following the same feature engineering approach as Solar and Wind Offshore.

In [ ]:
def create_time_features(df):
    """Create temporal features"""
    df = df.copy()
    
    # Basic time features
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['day_of_month'] = df.index.day
    df['day_of_year'] = df.index.dayofyear
    df['week_of_year'] = df.index.isocalendar().week
    df['month'] = df.index.month
    df['quarter'] = df.index.quarter
    df['year'] = df.index.year
    
    # Cyclical encoding
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    # Binary indicators
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['is_night'] = ((df['hour'] >= 22) | (df['hour'] <= 6)).astype(int)
    df['is_peak_hour'] = ((df['hour'] >= 7) & (df['hour'] <= 9) | 
                          (df['hour'] >= 17) & (df['hour'] <= 20)).astype(int)
    
    return df

df = create_time_features(df)
print(f"After time features: {df.shape}")

In [ ]:
def create_lag_features(df, target_col='price', lags=[1, 2, 3, 6, 12, 24, 48, 168]):
    """Create lag features"""
    df = df.copy()
    
    for lag in lags:
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    
    return df

df = create_lag_features(df)
print(f"After lag features: {df.shape}")

In [ ]:
def create_rolling_features(df, target_col='price', windows=[3, 6, 12, 24, 168]):
    """Create rolling statistical features"""
    df = df.copy()
    
    for window in windows:
        df[f'rolling_mean_{window}'] = df[target_col].shift(1).rolling(window=window).mean()
        df[f'rolling_std_{window}'] = df[target_col].shift(1).rolling(window=window).std()
        df[f'rolling_min_{window}'] = df[target_col].shift(1).rolling(window=window).min()
        df[f'rolling_max_{window}'] = df[target_col].shift(1).rolling(window=window).max()
    
    return df

df = create_rolling_features(df)
print(f"After rolling features: {df.shape}")

In [ ]:
def create_diff_features(df, target_col='price'):
    """Create difference features"""
    df = df.copy()
    
    df['diff_1'] = df[target_col].diff(1)
    df['diff_24'] = df[target_col].diff(24)  # Day-over-day
    df['diff_168'] = df[target_col].diff(168)  # Week-over-week
    
    return df

df = create_diff_features(df)
print(f"After diff features: {df.shape}")

In [ ]:
def create_price_specific_features(df, target_col='price'):
    """Create price-specific features"""
    df = df.copy()
    
    # Negative price indicator
    df['is_negative'] = (df[target_col] < 0).astype(int)
    
    # Price volatility (rolling std / rolling mean)
    df['volatility_24h'] = df[target_col].rolling(window=24).std() / (df[target_col].rolling(window=24).mean().abs() + 1e-8)
    
    # Price momentum
    df['momentum_3h'] = df[target_col] - df[target_col].shift(3)
    df['momentum_6h'] = df[target_col] - df[target_col].shift(6)
    
    return df

df = create_price_specific_features(df)
print(f"After price-specific features: {df.shape}")
print(f"\nTotal features created: {df.shape[1]}")

In [ ]:
# Check feature columns
print("All features:")
print(df.columns.tolist())

## 4. Handle Missing Values

In [ ]:
# Check missing values (from rolling/lag features)
print("Missing values per column:")
print(df.isnull().sum()[df.isnull().sum() > 0])

# Drop initial rows with NaN (from rolling windows)
print(f"\nBefore dropping NaN: {len(df)}")
df_clean = df.dropna()
print(f"After dropping NaN: {len(df_clean)}")
print(f"Rows dropped: {len(df) - len(df_clean)}")

## 5. Train/Val/Test Split

In [ ]:
# Define split dates
val_start = '2024-07-01'
test_start = '2024-10-01'

# Split data
train = df_clean[:val_start].copy()
val = df_clean[val_start:test_start].copy()
test = df_clean[test_start:].copy()

print("📊 Dataset Split:")
print(f"Train: {train.index.min()} to {train.index.max()} ({len(train)} hours)")
print(f"Val:   {val.index.min()} to {val.index.max()} ({len(val)} hours)")
print(f"Test:  {test.index.min()} to {test.index.max()} ({len(test)} hours)")
print(f"\nTotal: {len(df_clean)} hours")
print(f"Train: {len(train)/len(df_clean)*100:.1f}%")
print(f"Val:   {len(val)/len(df_clean)*100:.1f}%")
print(f"Test:  {len(test)/len(df_clean)*100:.1f}%")

## 6. Feature Scaling

In [ ]:
# Separate features and target
feature_cols = [col for col in df_clean.columns if col != 'price']
target_col = 'price'

print(f"Number of features: {len(feature_cols)}")

# Extract X and y
X_train = train[feature_cols]
y_train = train[target_col]

X_val = val[feature_cols]
y_val = val[target_col]

X_test = test[feature_cols]
y_test = test[target_col]

print(f"\nTrain: X={X_train.shape}, y={y_train.shape}")
print(f"Val:   X={X_val.shape}, y={y_val.shape}")
print(f"Test:  X={X_test.shape}, y={y_test.shape}")

In [ ]:
# Scale features (fit on train, transform all)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=feature_cols, index=X_val.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test.index)

print("✅ Features scaled using StandardScaler")

## 7. Save Processed Data

In [ ]:
# Create output directory
output_dir = Path('../../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# Combine X and y for saving
train_processed = X_train_scaled.copy()
train_processed['price'] = y_train

val_processed = X_val_scaled.copy()
val_processed['price'] = y_val

test_processed = X_test_scaled.copy()
test_processed['price'] = y_test

# Save to CSV
train_processed.to_csv(output_dir / 'price_train.csv')
val_processed.to_csv(output_dir / 'price_val.csv')
test_processed.to_csv(output_dir / 'price_test.csv')

print("✅ Processed data saved:")
print(f"   - {output_dir / 'price_train.csv'}")
print(f"   - {output_dir / 'price_val.csv'}")
print(f"   - {output_dir / 'price_test.csv'}")

In [ ]:
# Save feature names for later use
import json

feature_info = {
    'feature_columns': feature_cols,
    'n_features': len(feature_cols),
    'target_column': target_col,
    'train_size': len(train),
    'val_size': len(val),
    'test_size': len(test),
    'val_start': val_start,
    'test_start': test_start
}

with open(output_dir / 'price_feature_info.json', 'w') as f:
    json.dump(feature_info, f, indent=2)

print("\n✅ Feature info saved to price_feature_info.json")

## 8. Visualize Processed Data

In [ ]:
# Visualize train/val/test split
fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(y_train.index, y_train.values, linewidth=0.8, label='Train', alpha=0.7)
ax.plot(y_val.index, y_val.values, linewidth=0.8, label='Validation', alpha=0.7)
ax.plot(y_test.index, y_test.values, linewidth=0.8, label='Test', alpha=0.7)
ax.axhline(0, color='red', linestyle='--', linewidth=1)
ax.set_title('Processed Price Data - Train/Val/Test Split', fontweight='bold', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Price (EUR/MWh)')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../../results/figures/price_processed_split.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Feature correlation heatmap (sample)
sample_features = ['lag_1', 'lag_24', 'rolling_mean_24', 'rolling_std_24', 
                   'hour', 'day_of_week', 'is_weekend', 'diff_24', 'volatility_24h']
corr_df = train_processed[sample_features + ['price']].corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_df, annot=True, fmt='.2f', cmap='coolwarm', center=0, ax=ax)
ax.set_title('Feature Correlation Matrix (Sample)', fontweight='bold')
plt.tight_layout()
plt.savefig('../../results/figures/price_feature_correlation.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Summary

In [ ]:
print("="*80)
print("📋 PRICE DATA PREPROCESSING - SUMMARY")
print("="*80)

print(f"\n1. FEATURES CREATED:")
print(f"   - Total features: {len(feature_cols)}")
print(f"   - Time features: 19")
print(f"   - Lag features: 8")
print(f"   - Rolling features: 20 (5 windows × 4 stats)")
print(f"   - Difference features: 3")
print(f"   - Price-specific features: 5")

print(f"\n2. DATASET SPLIT:")
print(f"   - Train: {len(train)} hours ({len(train)/len(df_clean)*100:.1f}%)")
print(f"   - Val:   {len(val)} hours ({len(val)/len(df_clean)*100:.1f}%)")
print(f"   - Test:  {len(test)} hours ({len(test)/len(df_clean)*100:.1f}%)")

print(f"\n3. SPECIAL HANDLING:")
print(f"   - Negative prices: KEPT (valid oversupply indicator)")
print(f"   - Count in train: {(y_train < 0).sum()}")
print(f"   - Count in val: {(y_val < 0).sum()}")
print(f"   - Count in test: {(y_test < 0).sum()}")

print(f"\n4. DATA QUALITY:")
print(f"   - Missing values: 0 (dropped)")
print(f"   - Scaling: StandardScaler (fit on train)")
print(f"   - Ready for modeling: ✅")

print(f"\n5. FILES SAVED:")
print(f"   - price_train.csv: {train_processed.shape}")
print(f"   - price_val.csv: {val_processed.shape}")
print(f"   - price_test.csv: {test_processed.shape}")
print(f"   - price_feature_info.json")

print("\n" + "="*80)
print("✅ Preprocessing complete! Ready for baseline models.")
print("="*80)

## Next Steps

1. ✅ Data exploration complete
2. ✅ Data preprocessing complete
3. ➡️ **Next:** `03_price_baseline_models.ipynb`
   - Naive, Seasonal Naive, Moving Average, Drift, Mean
   - Quick benchmarks
4. 📊 Then: Statistical, ML, and Deep Learning models